### Neural Computation Paper Figures

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FormatStrFormatter
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap

### Parameters

In [ ]:
fontsize = 16
figsize = (10, 10)

In [ ]:
class lca_512_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_512_vh"
    self.display_name = "Sparse Coding"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False

class lca_768_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_768_vh"
    self.display_name = "Sparse Coding"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False

class lca_1024_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_1024_vh"
    self.display_name = "Sparse Coding"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False

class sae_768_vh_params(object):
  def __init__(self):
    self.model_type = "sae"
    self.model_name = "sae_768_vh"
    self.display_name = "Sparse Autoencoder"
    self.version = "1.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class rica_768_vh_params(object):
  def __init__(self):
    self.model_type = "rica"
    self.model_name = "rica_768_vh"
    self.display_name = "Linear Autoencoder"
    self.version = "0.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class lca_768_mnist_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_768_mnist"
    self.display_name = "Sparse Coding"
    self.version = "0.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class ae_768_mnist_params(object):
  def __init__(self):
    self.model_type = "ae"
    self.model_name = "ae_768_mnist"
    self.display_name = "ReLU Autoencoder"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class sae_768_mnist_params(object):
  def __init__(self):
    self.model_type = "sae"
    self.model_name = "sae_768_mnist"
    self.display_name = "Sparse Autoencoder"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class rica_768_mnist_params(object):
  def __init__(self):
    self.model_type = "rica"
    self.model_name = "rica_768_mnist"
    self.display_name = "Linear Autoencoder"
    self.version = "0.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class ae_deep_mnist_params(object):
  def __init__(self):
    self.model_type = "ae"
    self.model_name = "ae_deep_mnist"
    self.display_name = "ReLU Autoencoder"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

### Iso-contour activations comparison

In [ ]:
#params_list = [lca_768_vh_params(), sae_768_vh_params(), sae_768_vh_params(), rica_768_vh_params()]
params_list = [rica_768_mnist_params(), ae_768_mnist_params(), sae_768_mnist_params(), lca_768_mnist_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)
analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  analyzer.model.setup(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

In [ ]:
for analyzer in analyzer_list:
  run_params = np.load(analyzer.analysis_out_dir+"savefiles/iso_params_"+analyzer.analysis_params.save_info+".npz")["data"].item()
  min_angle = run_params["min_angle"]
  max_angle = run_params["max_angle"]
  num_neurons = run_params["num_neurons"]
  use_bf_stats = run_params["use_bf_stats"]
  num_comparison_vectors = run_params["num_comparison_vects"]
  use_random_orth_vects = run_params["use_random_orth_vects"]
  x_range = run_params["x_range"]
  y_range = run_params["y_range"]
  num_images = run_params["num_images"]
  analyzer.activations = np.load(analyzer.analysis_out_dir+"savefiles/iso_normalized_activations_"+analyzer.analysis_params.save_info+".npz")["data"]
  analyzer.contour_dataset = np.load(analyzer.analysis_out_dir+"savefiles/iso_contour_dataset_"+analyzer.analysis_params.save_info+".npz")["data"].item()

In [ ]:
analyzer_list[0].activations.shape

In [ ]:
neuron_index = [0,0,0,0]#[1, 1, 1, 1]
orth_index = [2, 2, 2, 2]
num_plots_y = 2
num_plots_x = 2
num_levels = 10

gs0 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=-0.1, hspace=0.5)
fig = plt.figure(figsize=(figsize[0]*1.5, figsize[1]*1))
cmap = plt.get_cmap("cividis")#"Greys_r")#"viridis")
vmin = np.min([analyzer.activations for analyzer in analyzer_list])
vmax = np.max([analyzer.activations for analyzer in analyzer_list])
levels = np.linspace(vmin, vmax, num_levels)
  
analyzer_index = 0
for plot_id in  np.ndindex((num_plots_y, num_plots_x)):
  (y_id, x_id) = plot_id
  if type(neuron_index) == list:
    analyzer_neuron_index = neuron_index[analyzer_index]
  else:
    analyzer_neuron_index = neuron_index
  if type(orth_index) == list:
    analyzer_orth_index = orth_index[analyzer_index]
  else:
    analyzer_orth_index = orth_index
  analyzer = analyzer_list[analyzer_index]
  gs1 = gridspec.GridSpecFromSubplotSpec(1, 2, gs0[plot_id], wspace=-0.15, hspace=0.1)
  curve_ax = pf.clear_axis(fig.add_subplot(gs1[0]))
  curve_ax.set_title(analyzer.analysis_params.display_name, fontsize=fontsize)

  norm_activity = analyzer.activations[analyzer_neuron_index, analyzer_orth_index, ...]
  contsf = curve_ax.contourf(analyzer.contour_dataset["X_mesh"], analyzer.contour_dataset["Y_mesh"], norm_activity,
    levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)

  proj_target = analyzer.contour_dataset["proj_target_neuron"][analyzer_neuron_index][analyzer_orth_index]
  curve_ax.arrow(0, 0, proj_target[0].item(), proj_target[1].item(),
    width=0.00, head_width=0.15, head_length=0.15, fc='k', ec='k',
    linestyle='-', linewidth=3)

  proj_comparison = analyzer.contour_dataset["proj_comparison_neuron"][analyzer_neuron_index][analyzer_orth_index]
  curve_ax.arrow(0, 0, proj_comparison[0].item(), proj_comparison[1].item(),
    width=0.00, head_width=0.15, head_length=0.15, fc='k', ec='k',
    linestyle="dotted", linewidth=2.0)

  #for proj_alt in analyzer.contour_dataset["proj_comparison_neuron"][analyzer_neuron_index]:
  #  if not np.all(proj_alt == proj_comparison):
  #    curve_ax.arrow(0, 0, proj_alt[0].item(), proj_alt[1].item(),
  #      width=0.00, head_width=0.15, head_length=0.15, fc='w', ec='w',
  #      linestyle="dashed", linewidth=1.0, alpha=0.9)

  proj_orth = analyzer.contour_dataset["proj_orth_vect"][analyzer_neuron_index][analyzer_orth_index]
  curve_ax.arrow(0, 0, proj_orth[0].item(), proj_orth[1].item(),
    width=0.00, head_width=0.10, head_length=0.10, fc='k', ec='k',
    linestyle="-", linewidth=3)

  curve_ax.plot(x_range, [0,0], color='k')
  curve_ax.plot([0,0], y_range, color='k')
  #curve_ax.set_xticks(x_range)
  #curve_ax.set_yticks(y_range)
  #curve_ax.tick_params(axis="both", bottom=True, top=False, left=True, right=False)
  #curve_ax.get_xaxis().set_visible(True)
  #curve_ax.get_yaxis().set_visible(True)
  #curve_ax.set_xticklabels(x_range, fontsize=fontsize)
  #curve_ax.set_yticklabels(x_range, fontsize=fontsize)

  #gs2 = gridspec.GridSpecFromSubplotSpec(2, 1, gs1[1], wspace=0.0, hspace=0.5)#-0.55)
  #target_vect_ax = pf.clear_axis(fig.add_subplot(gs2[0]))
  #target_vect_ax.imshow(analyzer.bf_stats["basis_functions"][analyzer.target_neuron_ids[0]], cmap="Greys_r")
  #target_vect_ax.set_title("Primary\nBasis Function", color='r', fontsize=16)
  #comparison_vect_ax = pf.clear_axis(fig.add_subplot(gs2[1]))
  #comparison_vect_ax.imshow(analyzer.bf_stats["basis_functions"][analyzer.comparison_neuron_ids[0][0]], cmap="Greys_r")
  #comparison_vect_ax.set_title("Comparison\nBasis Function", color='k', fontsize=16)

  analyzer_index += 1


plt.show()
for analyzer in analyzer_list:
  fig.savefig(analyzer.analysis_out_dir+"/vis/iso_contour_comparison"+analyzer.analysis_params.save_info+".eps")

### Orientation Selectivity

In [ ]:
params_list = [rica_768_vh_params(), sae_768_vh_params(), lca_768_vh_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)
analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  analyzer.model.setup(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

In [ ]:
min_rads = []
max_rads = []
for analyzer in analyzer_list:
  analyzer.bf_spatial_freq_rads = [np.sqrt(x**2+y**2) for (y,x) in analyzer.bf_stats["fourier_centers"]]
  min_rads.append(np.min(analyzer.bf_spatial_freq_rads))
  max_rads.append(np.max(analyzer.bf_spatial_freq_rads))
  
num_bins = 10
min_rad = np.min(min_rads)
max_rad = np.max(max_rads)
bins = np.linspace(min_rad, max_rad, num_bins)
fig, ax = plt.subplots(1, figsize=figsize)
hist_max = []
for analyzer in analyzer_list:
  hist, bin_edges = np.histogram(analyzer.bf_spatial_freq_rads, bins, density=True)
  bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
  bin_centers = bin_left + (bin_right - bin_left)/2
  
  label = analyzer.model.params.model_type.upper() + " " + str(analyzer.model.get_num_latent())# + " van Hateren"
  #label = re.sub("_", " ", analyzer.model_name)
  ax.plot(bin_centers, hist, alpha=1.0, linestyle="-", drawstyle="steps-mid", label=label)
  hist_max.append(np.max(hist))
  
ax.set_xticks(bin_left, minor=True)
ax.set_xticks(bin_left[::4], minor=False)
ax.xaxis.set_major_formatter(FormatStrFormatter("%0.0f"))
ax.tick_params("both", labelsize=16)
ax.set_xlim([min_rad, max_rad])
ax.set_xticks([0, int(np.floor(max_rad/4)), int(2*np.floor(max_rad/4)),
  int(3*np.floor(max_rad/4)), max_rad])
ax.set_ylim([0, 0.6])
ax.set_xlabel("Spatial Frequency", fontsize=fontsize)
ax.set_ylabel("Normalized Density", fontsize=fontsize)
ax.set_title("Neuron Weight Spatial Frequency Histogram", fontsize=fontsize)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, labels, fontsize=fontsize, ncol=3,
  borderaxespad=0., bbox_to_anchor=[0.01, 0.99], fancybox=True, loc="upper left")
for line in legend.get_lines():
  line.set_linewidth(3)
plt.show()    

In [ ]:
angle_heatmap_fig = pf.plot_weight_angle_heatmap(analyzer.plot_matrix, angle_min=0, angle_max=180,
  title="Angles Between Neurons", figsize=(8,8))

### DRAE Iso-Response Contours

In [ ]:
params_list = [ae_deep_mnist_params()]#,lca_768_vh_params()]#, rica_768_vh_params(), sae_768_vh_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)
analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  analyzer.model.setup(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

In [ ]:
min_angle = 20
max_angle = 55
num_neurons = 2 # How many neurons to plot
use_bf_stats = True # If false, then use optimal stimulus
num_comparison_vects = 10
use_random_orth_vects = False
x_range = [-2, 2]
y_range = [-2, 2]
num_images = int(10**2)

for analyzer in analyzer_list:
  compute_iso_vectors(analyzer, min_angle, max_angle, num_neurons, use_bf_stats)
  contour_dataset = get_contour_dataset(analyzer, num_comparison_vects,
    use_random_orth_vects, x_range, y_range, num_images)
  analyzer.activations = get_normalized_activations(analyzer, contour_dataset["datapoints"])
  contour_dataset.pop("datapoints")
  analyzer.contour_dataset = contour_dataset

In [ ]:
num_plots_y = num_neurons + 1 # extra dimension for example image
num_plots_x = num_neurons + 1 # extra dimension for example image

gs0 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.1, hspace=0.1)
fig = plt.figure(figsize=(10, 10))
cmap = plt.get_cmap('viridis')

orth_vectors = []
for neuron_loop_index in range(num_neurons): # rows
  for orth_loop_index in range(num_neurons): # columns
    norm_activity = analyzer.activations[neuron_loop_index][orth_loop_index]
    proj_target = analyzer.contour_dataset["proj_target_neuron"][neuron_loop_index][orth_loop_index]
    proj_comparison = analyzer.contour_dataset["proj_comparison_neuron"][neuron_loop_index][orth_loop_index]
    proj_orth = analyzer.contour_dataset["proj_orth_vect"][neuron_loop_index][orth_loop_index]
    orth_vectors.append(analyzer.contour_dataset["orth_vect"][neuron_loop_index][orth_loop_index])

    curve_plot_y_idx = neuron_loop_index + 1
    curve_plot_x_idx = orth_loop_index + 1
    curve_ax = pf.clear_axis(fig.add_subplot(gs0[curve_plot_y_idx, curve_plot_x_idx]))

    # NOTE: each subplot has a renormalized color scale
    # TODO: Add scale bar like in the lca inference plots
    vmin = np.min(norm_activity)
    vmax = np.max(norm_activity)

    levels = 5
    contsf = curve_ax.contourf(analyzer.contour_dataset["X_mesh"], analyzer.contour_dataset["Y_mesh"], norm_activity,
      levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)

    curve_ax.arrow(0, 0, proj_target[0].item(), proj_target[1].item(),
      width=0.05, head_width=0.15, head_length=0.15, fc='r', ec='r')
    curve_ax.arrow(0, 0, proj_comparison[0].item(), proj_comparison[1].item(),
      width=0.05, head_width=0.15, head_length=0.15, fc='w', ec='w')
    curve_ax.arrow(0, 0, proj_orth[0].item(), proj_orth[1].item(),
      width=0.05, head_width=0.15, head_length=0.15, fc='k', ec='k')

    curve_ax.set_xlim(x_range)
    curve_ax.set_ylim(y_range)
    
for plot_y_id in range(num_plots_y):
  for plot_x_id in range(num_plots_x):
    if plot_y_id > 0 and plot_x_id == 0:
      bf_ax = pf.clear_axis(fig.add_subplot(gs0[plot_y_id, plot_x_id]))
      bf_resh = analyzer.target_vectors[plot_y_id-1].reshape((int(np.sqrt(np.prod(analyzer.model.params.data_shape))),
        int(np.sqrt(np.prod(analyzer.model.params.data_shape)))))
      bf_ax.imshow(bf_resh, cmap="Greys_r")
      if plot_y_id == 1:
        bf_ax.set_title("Target vectors", color="r", fontsize=16)
    if plot_y_id == 0 and plot_x_id > 0:
      orth_img = orth_vectors[plot_x_id-1].reshape(int(np.sqrt(np.prod(analyzer.model.params.data_shape))),
        int(np.sqrt(np.prod(analyzer.model.params.data_shape))))
      orth_ax = pf.clear_axis(fig.add_subplot(gs0[plot_y_id, plot_x_id]))
      orth_ax.imshow(orth_img, cmap="Greys_r")
      if plot_x_id == 1:
        orth_ax.set_title("Orthogonal vectors", color="k", fontsize=16)

plt.show()
#fig.savefig(analyzer.analysis_out_dir+"/vis/iso_contour_grid_04.png")